# Tableau and player form

Here we create a csv file that is used for making further Tableau visualizations. 

In [1]:
# recent gameweek
gameweek = 30

In [2]:
# import basic libraries
import pandas as pd
import numpy as np

pd.set_option('max_columns',100)

In [3]:
# fetch player stats data
all_df = pd.read_csv('../data/fpl/data_week1.csv', index_col=0)
all_df['gameweek'] = 1
all_df['xPoints'] = all_df['adjusted points']

for week in range(2,gameweek+1):
    filepath = '../data/fpl/data_week' + str(week) + '.csv'
    df = pd.read_csv(filepath, index_col=0)
    df['gameweek'] = week
    column_name = 'xPoints week ' + str(week)
    df['xPoints'] = df[column_name]
    all_df = all_df.append(df,sort='False')

all_df.shape

(18739, 300)

In [4]:
# calculate player form
for i in [5,10,15,20]:
    gameweeks = np.arange(gameweek-i+1,gameweek+1)
    column_name = 'form ' + str(i)
    all_df.loc[all_df['gameweek']==gameweek, column_name] = (all_df[all_df['gameweek'].isin(gameweeks)].groupby(['id'])['xPoints'].sum() / (all_df.loc[(all_df['gameweek']==gameweek),'games played'].round() - all_df.loc[(all_df['gameweek']==gameweek-i),'games played'].round()) )
    # dirty fix in case there are inf results
    all_df.loc[(all_df['gameweek']==gameweek) & np.isinf(all_df[column_name]), column_name] = np.nan

In [5]:
# top form players
all_df[all_df['gameweek']==gameweek][['web_name', 'games played','total_points', 'points_per_game','adjusted points','adjusted points per game',\
                                'form 5', 'form 10', 'form 15', 'form 20']].sort_values(by=['form 10'], ascending=False).head(20)

,web_name,games played,total_points,points_per_game,adjusted points,adjusted points per game,form 5,form 10,form 15,form 20
id,,,,,,,,,,
112,Arrizabalaga,3.947368,15,3.8,17.571071,4.451338,NaN,6.986341,6.986341,6.986341
388,Kane,27.945205,204,7.3,182.075346,6.515441,7.486651,6.691101,6.729817,6.178366
239,Justin,22.954545,101,4.4,90.031958,3.922184,NaN,6.618239,5.232324,4.979129
302,Fernandes,30.140845,214,7.1,193.798937,6.429778,4.823887,6.309027,6.136798,6.448270
506,Wilson,21.090909,116,5.5,103.724006,4.917949,NaN,5.765906,3.744051,4.016907
245,Adrián,2.972973,11,3.7,10.617766,3.571431,5.730734,5.730734,5.730734,5.730734
104,Alonso,10.000000,42,4.2,44.354047,4.435405,6.315901,5.729648,5.729648,5.729648
233,Iheanacho,16.764706,57,3.4,56.320000,3.359439,6.860000,5.628571,4.620000,3.830769
570,Raphinha,25.111111,113,4.5,113.371349,4.514788,5.419014,5.622545,5.176445,5.005846


In [6]:
# save data
filepath = '../data/fpl/all_data.csv'
all_df.to_csv(filepath)

filepath = '../data/fpl/data_week' + str(gameweek) + '.csv'
all_df[all_df['gameweek']==gameweek].to_csv(filepath)